Deep Learning Script for Classifying Stimulus Bbased on EEG Data

In [1]:
# Import necessary libraries
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization,
                                     Activation, MaxPooling2D, Dropout, Flatten, Dense)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# Suppress TensorFlow warnings (optional)
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)


In [2]:

# Data Preparation
# Define the data directory where subject folders are located
data_dir = '/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed'

# List all subject folders
subject_folders = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]

# Initialize lists to hold data and labels from all subjects
X_list = []
y_list = []

# Loop over each subject folder
for subject in subject_folders:
    subject_data_dir = os.path.join(data_dir, subject)
    # Construct file paths for the subject's data
    X_file = os.path.join(subject_data_dir, f'Epochs_{subject}.fif')
    y_file = os.path.join(subject_data_dir, f'y_labels_{subject}.npy')
    
    # Check if data files exist
    if os.path.exists(X_file) and os.path.exists(y_file):
        print(f'Loading data for {subject}')
        # Load the data
        X_subject = mne.read_epochs(X_file, preload=True, verbose=False)
        y_subject = np.load(y_file)
        # Append to the list
        X_sub_data = X_subject.get_data()
        X_list.append(X_sub_data)
        y_list.append(y_subject)
    else:
        print(f'Data files not found for {subject}')

# Ensure that at least one subject has been loaded
if len(X_list) == 0:
    raise ValueError("No data was loaded. Please check your data directory and files.")

# Get channel names from the last loaded subject
channels = X_subject.ch_names
coi = ['P8', 'PO8', 'O2', 'P10']
chan_idx = [channels.index(chan) for chan in coi]


Loading data for sub-021


/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-021/Epochs_sub-021.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)
/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-026/Epochs_sub-026.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)


Loading data for sub-026
Loading data for sub-019
Loading data for sub-010


/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-019/Epochs_sub-019.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)
/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-010/Epochs_sub-010.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)
/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-017/Epochs_sub-017.fif) does not conform to MNE naming conve

Loading data for sub-017
Loading data for sub-028
Loading data for sub-016


...

/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-030/Epochs_sub-030.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)
/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-031/Epochs_sub-031.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)


Loading data for sub-009
Loading data for sub-036


/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-009/Epochs_sub-009.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)
/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-036/Epochs_sub-036.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)


Loading data for sub-038
Loading data for sub-007


/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-038/Epochs_sub-038.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)
/var/folders/dh/551p960n6mv9t9byzj8pp0640000gp/T/ipykernel_16542/3929881324.py:23: RuntimeWarning: This filename (/Users/BAEK/Code/neurEx/data/N170/Data_Preprocessed/sub-007/Epochs_sub-007.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  X_subject = mne.read_epochs(X_file, preload=True, verbose=False)


In [4]:

# Concatenate data from all subjects
X = np.concatenate(X_list, axis=0)
X = X[:, chan_idx, :]  # Select channels of interest
y = np.concatenate(y_list, axis=0)

print(f'Combined data shape before filtering: X={X.shape}, y={y.shape}')

# Filter stimulus events
stimulus_labels = [1, 2]  # 1: Face, 2: Car
stimulus_mask = np.isin(y, stimulus_labels)
X = X[stimulus_mask]
y = y[stimulus_mask]

# Adjust labels to start from 0
y = y - 1

# Focus on the N170 response by selecting data around 170 ms
tmin, tmax = 0.1, 0.2  # 100 ms to 200 ms
times = X_subject.times  # Time vector from the epochs
time_mask = (times >= tmin) & (times <= tmax)
X_focused = X[:, :, time_mask]

print(f'Combined data shape after filtering: X={X.shape}, y={y.shape}')

X_focused[0,0]

Combined data shape before filtering: X=(23602, 4, 820), y=(23602,)
Combined data shape after filtering: X=(6104, 4, 820), y=(6104,)


array([-2.65535473e-07, -2.33084976e-07, -2.56221681e-07, -3.35722922e-07,
       -4.71088408e-07, -6.60497783e-07, -9.00880692e-07, -1.18786204e-06,
       -1.51586949e-06, -1.87812459e-06, -2.26676702e-06, -2.67311763e-06,
       -3.08767533e-06, -3.50033306e-06, -3.90088247e-06, -4.27891778e-06,
       -4.62421035e-06, -4.92699383e-06, -5.17809914e-06, -5.36951111e-06,
       -5.49421166e-06, -5.54619740e-06, -5.52102088e-06, -5.41581248e-06,
       -5.22933673e-06, -4.96202801e-06, -4.61585473e-06, -4.19435119e-06,
       -3.70253657e-06, -3.14671301e-06, -2.53441977e-06, -1.87421798e-06,
       -1.17549646e-06, -4.48325484e-07,  2.96874105e-07,  1.04961514e-06,
        1.79936683e-06,  2.53584337e-06,  3.24924111e-06,  3.93041848e-06,
        4.57091855e-06,  5.16319560e-06,  5.70080136e-06,  6.17847393e-06,
        6.59201477e-06,  6.93824957e-06,  7.21520756e-06,  7.42191790e-06,
        7.55842064e-06,  7.62581203e-06,  7.62584541e-06,  7.56124113e-06,
        7.43529985e-06,  

In [5]:
print(X_focused.shape)

X_focused

(6104, 4, 102)


array([[[-2.65535473e-07, -2.33084976e-07, -2.56221681e-07, ...,
          2.21065354e-06,  2.27832317e-06,  2.31764697e-06],
        [ 1.52646923e-06,  1.55423677e-06,  1.55282390e-06, ...,
          7.41099594e-07,  9.07904801e-07,  1.05732774e-06],
        [-7.22147654e-06, -7.38670919e-06, -7.53458689e-06, ...,
         -4.07241284e-07, -2.46243744e-07, -8.30428524e-08],
        [ 5.03884314e-06,  5.25305508e-06,  5.36733101e-06, ...,
          8.15229414e-06,  8.16074655e-06,  8.08678148e-06]],

       [[-1.47737646e-06, -1.67229044e-06, -1.82316827e-06, ...,
          3.79423474e-06,  3.87306189e-06,  3.89641451e-06],
        [-6.68424724e-07, -6.97640298e-07, -7.08379923e-07, ...,
         -1.09889269e-06, -8.65716336e-07, -6.51313899e-07],
        [-7.48422084e-07, -9.07281396e-07, -1.03889632e-06, ...,
         -8.53871045e-07, -6.54174386e-07, -4.62437718e-07],
        [-4.21238231e-06, -4.83159445e-06, -5.40723513e-06, ...,
         -1.00315418e-08,  2.54398673e-07,  5.16855

In [8]:

# Normalize data per channel
def normalize_per_channel(X_data):
    X_norm = np.zeros_like(X_data)
    for i in range(X_data.shape[1]):  # Loop over channels
        scaler = StandardScaler()
        X_channel = X_data[:, i, :]
        X_norm[:, i, :] = scaler.fit_transform(X_channel)
    return X_norm

X = normalize_per_channel(X_focused)

# Reshape data for Conv2D (samples, channels, times, 1)
X = X.transpose((0, 1, 2))  # Shape: (samples, channels, times)
X = X[..., np.newaxis]  # Shape: (samples, channels, times, 1)

# Data Augmentation
def augment_data(X, y):
    X_augmented = []
    y_augmented = []
    for i in range(X.shape[0]):
        # Original data
        X_augmented.append(X[i])
        y_augmented.append(y[i])

        # Time-shifted data
        for shift in [-2, -1, 1, 2]:  # Shift by 1 or 2 time steps
            X_shifted = np.roll(X[i], shift, axis=2)
            X_augmented.append(X_shifted)
            y_augmented.append(y[i])

        # Noise-injected data
        noise = np.random.normal(0, 0.01, X[i].shape)
        X_noisy = X[i] + noise
        X_augmented.append(X_noisy)
        y_augmented.append(y[i])

    return np.array(X_augmented), np.array(y_augmented)

# Apply augmentation on the whole dataset before splitting
X_aug, y_aug = augment_data(X, y)

print(f'Augmented data shape: X={X_aug.shape}, y={y_aug.shape}')

X_aug


Augmented data shape: X=(36624, 4, 102, 1), y=(36624,)


array([[[[-0.31605341],
         [-0.32619682],
         [-0.34412525],
         ...,
         [ 0.32308469],
         [ 0.31376044],
         [ 0.30061452]],

        [[-0.11632468],
         [-0.12962838],
         [-0.14648719],
         ...,
         [-0.11461243],
         [-0.11596127],
         [-0.11936317]],

        [[-0.99076982],
         [-1.01335294],
         [-1.03497392],
         ...,
         [-0.41966924],
         [-0.41317054],
         [-0.40664085]],

        [[ 0.28014587],
         [ 0.29523614],
         [ 0.29830576],
         ...,
         [ 0.88123175],
         [ 0.86926358],
         [ 0.84805751]]],


       [[[-0.31605341],
         [-0.32619682],
         [-0.34412525],
         ...,
         [ 0.32308469],
         [ 0.31376044],
         [ 0.30061452]],

        [[-0.11632468],
         [-0.12962838],
         [-0.14648719],
         ...,
         [-0.11461243],
         [-0.11596127],
         [-0.11936317]],

        [[-0.99076982],
         [-1.0

In [ ]:

# Split data into training and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_aug, y_aug, test_size=0.2, random_state=42, stratify=y_aug)

# Model Implementation using DeepConvNet
def DeepConvNet(nb_classes, Chans=4, Samples=50, dropoutRate=0.5):
    input_main = Input((Chans, Samples, 1))
    
    # Block 1
    block1 = Conv2D(25, (1, 5), padding='same', use_bias=False)(input_main)
    block1 = Conv2D(25, (Chans, 1), use_bias=False)(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = MaxPooling2D((1, 2))(block1)
    block1 = Dropout(0.2)(block1)
    
    # Block 2
    block2 = Conv2D(50, (1, 5), padding='same', use_bias=False)(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = MaxPooling2D((1, 2))(block2)
    block2 = Dropout(0.3)(block2)
    
    # Block 3
    block3 = Conv2D(100, (1, 5), padding='same', use_bias=False)(block2)
    block3 = BatchNormalization()(block3)
    block3 = Activation('elu')(block3)
    block3 = MaxPooling2D((1, 2))(block3)
    block3 = Dropout(0.4)(block3)
    
    # Block 4
    block4 = Conv2D(200, (1, 5), padding='same', use_bias=False)(block3)
    block4 = BatchNormalization()(block4)
    block4 = Activation('elu')(block4)
    block4 = MaxPooling2D((1, 2))(block4)
    block4 = Dropout(0.5)(block4)
    
    # Flatten and Dense Layers
    flatten = Flatten()(block4)
    dense = Dense(nb_classes)(flatten)
    softmax = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

# Parameters for DeepConvNet
Chans = X.shape[1]
Samples = X.shape[2]
nb_classes = 2  # Binary classification

# Compile the model
model = DeepConvNet(nb_classes=nb_classes, Chans=Chans, Samples=Samples, dropoutRate=0.5)
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Adjust EarlyStopping and ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10,
                              min_lr=1e-6, verbose=1)

# Training
epochs = 500
batch_size = 16  # Keep batch size small for better gradient estimation

# Train the model
history = model.fit(X_train_val, y_train_val,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, reduce_lr],
                    verbose=1)

# Evaluation and Visualization
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Visualize training history
# Plot accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', color='blue')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Predictions
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Confusion Matrix and Classification Report
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Face', 'Car'],
            yticklabels=['Face', 'Car'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_test, y_pred, target_names=['Face', 'Car']))


In [88]:
model.save('/Users/owenanderson/Documents/NeurEx/Projects/P1_N170/Models/V2.keras')